In [285]:
import requests
import json
from IPython.core.display import display, HTML

Set the drug that we want to find associations for

In [286]:
myDrug = "INFLIXIMAB"

Make the API call to find all the drugs

In [287]:
esURL = 'http://172.17.7.25:80/16.08_evidence-data-generic/_search'

query = json.dumps({
        "fields": ["disease.efo_info.label", "drug.molecule_name", "drug.id", "target.gene_info.symbol"],
        "query": {
            "match": {
                "drug.molecule_name": myDrug
            }
        }
    })

response = requests.get(esURL, data=query)
output = response.json()

#print out the json for debug
print json.dumps(output, indent=2)

{
  "hits": {
    "hits": [
      {
        "_type": "evidencestring-chembl", 
        "_routing": "ENSG00000232810", 
        "_index": "16.08_evidence-data-generic", 
        "_score": 10.677767, 
        "fields": {
          "target.gene_info.symbol": [
            "TNF"
          ], 
          "disease.efo_info.label": [
            "Crohn's disease"
          ], 
          "drug.molecule_name": [
            "INFLIXIMAB"
          ], 
          "drug.id": [
            "http://identifiers.org/chembl.compound/CHEMBL1201581"
          ]
        }, 
        "_id": "4107b67547bcc8776bd0e6f4db1b384e"
      }, 
      {
        "_type": "evidencestring-chembl", 
        "_routing": "ENSG00000232810", 
        "_index": "16.08_evidence-data-generic", 
        "_score": 10.677767, 
        "fields": {
          "target.gene_info.symbol": [
            "TNF"
          ], 
          "disease.efo_info.label": [
            "colitis"
          ], 
          "drug.molecule_name": [
           

Collect information about diseases and targets associated with our drug

In [388]:

for data in output['hits']['hits']:
    
    #print data['evidence']['drug']['max_phase_for_all_diseases']['numeric_index']
    #print json.dumps(data, indent=2)

    for x in data['fields']['disease.efo_info.label']:
        diseases[x] = 'true'
    for y in data['fields']['target.gene_info.symbol']:
        targets[y] = 'true'

print "DRUG: " + myDrug

print "\nDISEASES: "

for key in diseases.keys():
    print "\t" + key
    
print "\nTARGETS:"

for key in targets.keys():
    print "\t" + key
    #display(HTML('<h1>' + key + '</h1>'))


DRUG: INFLIXIMAB

DISEASES: 
	acute graft vs. host disease
	eye disease
	skeletal system disease
	Crohn's disease
	rheumatoid arthritis
	immune system disease
	ankylosing spondylitis
	skin disease
	cardiovascular disease
	nervous system disease
	spondyloarthropathy
	digestive system disease
	colitis

TARGETS:
	TNF


In [386]:
data = """
            { label: "Care", groups: [{ label:"self" },{ label:"health" },{ label:"intervention" },{ label:"adherence" }], weight: 1.0 },
            { label: "Ci", groups: [{ label:"95" },{ label:"risk" },{ label:"gdm" },{ label:"woman" },{ label:"gestational" },{ label:"pregnancy" },{ label:"hr" },{ label:"associate" }], weight: 1.0 },
            { label: "Cell", groups: [{ label:"mouse" },{ label:"rat" },{ label:"insulin" },{ label:"diabetic" },{ label:"expression" },{ label:"induce" },{ label:"protein" },{ label:"glucose" },{ label:"islet" }], weight: 1.0 },
            { label: "Risk", groups: [{ label:"diesase" },{ label:"diabetic" },{ label:"high" },{ label:"age" },{ label:"associate" },{ label:"factor" }],weight: 1.0 },
            { label: "Insulin", groups: [{ label:"glucose" },{ label:"group" },{ label:"control" },{ label:"level" },{ label:"weight" },{ label:"treatment" }],weight: 1.0 }
        """

In [387]:
display(HTML("""
<!DOCTYPE html>
<html>
  <head>
    <title>FoamTree Quick Start</title>
    <meta charset="utf-8" />
  </head>

  <body>
    <div id="visualization" style="width: 800px; height: 600px"></div>

    <script src="carrotsearch.foamtree.js"></script>
    <script>
      function init() {
        var foamtree = new CarrotSearchFoamTree({
          id: "visualization",
          dataObject: {
            groups:["""
             + data +
             """
          ]}
        });
      }
        
    init();
    </script>
  </body>
 """))
